#Capstone Project: Best College Towns

##Introduction

As a web-based service that assists students in selecting schools, we accept requests from prospective high school students to compare colleges and universities along a variety of factors.  As high school students prepare to apply to colleges, they want to consider their college experience.  What type of college town are their target schools based in? They can't afford to visit all.  Therefore, we will leverage the Foursquare API and develop a K-means clustering to get some sense of what types of environments exist and how similar different schools are.  

We receive a request from a students with the schools they are interested in.  The ten schools the student is considering are: Northwestern University (Evanston, Illinois); University of Michigan (Ann Arbor, Michigan); Stanford (Palo Alto, California); Auburn (Auburn, Alabama); Florida State (Talllahasse, Florida); Wellesey College (Wellesey, Massachusetts); Tufts (Medford, Massachusetts); Columbia University (New York City, New York); Duke University (Durham, North Carolina); Washington University (St. Louis, Missouris).

##Data

We will use the Foursquare API and the school address to pull the top venues around with a 1 kilometer radius of the school, easily within walking distance.  The Foursquare data is regularly updated, high quality data set for understanding the types of venues that exist whether it be coffee shops, bars, or parks.

To use this data, we will take the school addresses and use the geopy.goecoders package to convert them into latitudes and longitudes for each school.  We will then retrieve the nearby venues from the API.  From the data, we hope to be able to give the student a profile of the different types of venues that are most present in the vicity of the school, how urban or rural the school is based on the number of venues in the area, as well as what types of schools are similar and different.